The code is below to get the information requested.

A couple of trends that I noticed from analyzing the data
1) A schools average math and reading scores stayed consistent throughout students time in school mean 9th through 12 grade. Schools didn't tend to improve or get worse, so maybe a students abilities coming out of middle school is a strong predictor of overall high school success. 

2) There appears to be a sweet spot for per student spending since overall pass rate increases before decreasing.  This could indicated that spending more per student is beneficial, but money doesn't solve anything.  The highest funded are probably the lowest performing in attempts to help.

3) School size doesn't appear to have a significant impact on student overall performance.  It would be nice to have the number of teachers so that we could do analysis on teacher/student ratio

In [128]:
#import libraries
import pandas as pd
import numpy as np

In [129]:
#create vars for the file paths
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

# create data frames for the data in the 2 files
school_df = pd.read_csv(school_data)
students_df = pd.read_csv(student_data)

#combine the school and student data frames into a new data frame called comb_df
comb_df = school_df.merge(students_df,on = "school_name")

In [151]:
#THIS SECTION WILL FIND DISTRICT SUMMARY RESULTS

# Create dictionary to hold District Summary details
dist_summ = {"Total Schools":0,
             "Total Students":0,
             "Total Budget":0,
             "Average Math Score":0.00,
             "Average Reading Score":0.00,
             "Percent Passing Math":0.00,
            "Percent Passing Reading":0.00,
            "Percent Overall Passing":0.00}

#Get the number of total schools, students, and budget from school_df,format, and store in district summary dictionary
dist_summ["Total Schools"] = school_df["school_name"].count()
dist_summ["Total Students"] = '{:,}'.format(students_df["Student ID"].count())
dist_summ["Total Budget"] ='${:,}'.format(school_df["budget"].sum())
#get the average math score and reading score from students_df, format, and store in district summary dictionary
dist_summ["Average Math Score"]='{:.2f}%'.format(students_df["math_score"].mean())
dist_summ["Average Reading Score"]='{:.2f}%'.format(students_df["reading_score"].mean())
# find the percent of students passing math/reading by dividing the total students that passed math/reading by total students
dist_summ["Percent Passing Math"]='{:.2f}%'.format(comb_df[comb_df["math_score"] >= 70]["Student ID"].count()/comb_df["Student ID"].count()*100)
dist_summ["Percent Passing Reading"]='{:.2f}%'.format(comb_df[comb_df["reading_score"] >= 70]["Student ID"].count()/comb_df["Student ID"].count()*100)
#% passing both would be the student that passed math and reading divided by total students
dist_summ["Percent Overall Passing"] = '{:.2f}%'.format((comb_df[(comb_df["reading_score"]>=70) & (comb_df["math_score"] >= 70)]["Student ID"].count())/comb_df["Student ID"].count()*100)

#Display District Summary Results
for row in dist_summ:
    print(row + ": " +str(dist_summ[row]))

Total Schools: 15
Total Students: 39,170
Total Budget: $24,649,428
Average Math Score: 78.99%
Average Reading Score: 81.88%
Percent Passing Math: 74.98%
Percent Passing Reading: 85.81%
Percent Overall Passing: 65.17%


In [152]:
#THIS SECTION WILL FIND SCHOOL SUMMARY INFO

#Create data frame to hold school summary information
school_summary_df = school_df[["school_name","type","size","budget"]]

#Calc per student budget by dividing budget by size and formatting
school_summary_df["Per Student Budget"]=(school_summary_df["budget"]/school_summary_df["size"])

#Rename the Column Headers
school_summary_df = school_summary_df.rename(columns = {'school_name':'School Name','type':'School Type','size':'Total Students','budget':'Total School Budget'})

#group df by school
school_grouped_df = comb_df.groupby("school_name")

#create 2 df's for average math score by school and average reading score by school
m_average_df = school_grouped_df[['math_score']].mean()
r_average_df = school_grouped_df[['reading_score']].mean()

#store those values in an arrays
school_m_avg = m_average_df.values
school_r_avg = r_average_df.values

#add those array to the school summary df
school_summary_df["Average Math Score"]=school_m_avg.round()
school_summary_df["Average Reading Score"]=school_r_avg.round()

#Create 3 dataframes 1) only include students that passed math 2) only include students that passed reading
# 3) only include students that passed both math and reading
pass_math_df = comb_df[comb_df["math_score"] >= 70]
pass_reading_df = comb_df[comb_df["reading_score"] >= 70]
pass_overall_df = comb_df[(comb_df["reading_score"] >= 70) & (comb_df["math_score"] >= 70)]


#Create new data frames of passing students that are grouped by school name
math_pass_by_school = pass_math_df.groupby("school_name")
reading_pass_by_school = pass_reading_df.groupby("school_name")
overall_pass_by_school = pass_overall_df.groupby("school_name")
#create new dataframe that shows the percent passed by school
per_pass_read_df = reading_pass_by_school["School ID"].count()/school_grouped_df["Student ID"].count()*100
#store those values as an array so we can add them to our School Summary df
per_pass_read = per_pass_read_df.values
#repeat steps for reading and overall
per_pass_math_df = math_pass_by_school["School ID"].count()/school_grouped_df["Student ID"].count()*100
per_pass_math = per_pass_math_df.values
per_pass_overall_df = overall_pass_by_school["School ID"].count()/school_grouped_df["Student ID"].count()*100
per_pass_overall = per_pass_overall_df.values

# add the columns to the dataframe
school_summary_df["Percent Passing Math"]= per_pass_math
school_summary_df["Percent Passing Reading"]= per_pass_read
school_summary_df["Percent Overall Passing"]=per_pass_overall

#Create a new df that I can format so that I can utilized the school_summary df for other activities
formatted_school_summ_df = school_summary_df.copy()


#Format output: looked up changing the format  source: https://stackoverflow.com/questions/35019156/pandas-format-column-as-currency

formatted_school_summ_df["Total School Budget"]=formatted_school_summ_df["Total School Budget"].apply(lambda x: "${:.2f}".format(x))
formatted_school_summ_df["Total Students"]=formatted_school_summ_df["Total Students"].apply(lambda x: "{:,}".format(x))
formatted_school_summ_df["Per Student Budget"] = formatted_school_summ_df["Per Student Budget"].apply(lambda x: "${:.2f}".format(x))
formatted_school_summ_df["Percent Passing Math"]=formatted_school_summ_df["Percent Passing Math"].apply(lambda x: "{:.2f}%".format(x))
formatted_school_summ_df["Percent Passing Reading"]=formatted_school_summ_df["Percent Passing Reading"].apply(lambda x: "{:.2f}%".format(x))
formatted_school_summ_df["Percent Overall Passing"]=formatted_school_summ_df["Percent Overall Passing"].apply(lambda x: "{:.2f}%".format(x))
formatted_school_summ_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
0,Huang High School,District,"2,917",$1910635.00,$655.00,77.0,81.0,66.68%,81.93%,54.64%
1,Figueroa High School,District,"2,949",$1884411.00,$639.00,83.0,84.0,94.13%,97.04%,91.33%
2,Shelton High School,Charter,"1,761",$1056600.00,$600.00,77.0,81.0,65.99%,80.74%,53.20%
3,Hernandez High School,District,"4,635",$3022020.00,$652.00,77.0,81.0,68.31%,79.30%,54.29%
4,Griffin High School,Charter,"1,468",$917500.00,$625.00,83.0,84.0,93.39%,97.14%,90.60%
5,Wilson High School,Charter,"2,283",$1319574.00,$578.00,77.0,81.0,66.75%,80.86%,53.53%
6,Cabrera High School,Charter,"1,858",$1081356.00,$582.00,84.0,84.0,92.51%,96.25%,89.23%
7,Bailey High School,District,"4,976",$3124928.00,$628.00,77.0,81.0,65.68%,81.32%,53.51%
8,Holden High School,Charter,427,$248087.00,$581.00,77.0,81.0,66.06%,81.22%,53.54%
9,Pena High School,Charter,962,$585858.00,$609.00,84.0,84.0,94.59%,95.95%,90.54%


In [148]:
# Sort the data frame by Percent Overall Passing to display the Top 5 Performing Schools
top_schools_df = formatted_school_summ_df.sort_values("Percent Overall Passing", ascending=False).head(5)
top_schools_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
1,Figueroa High School,District,"2,949",$1884411.00,$639.00,83.0,84.0,94.13%,97.04%,91.33%
12,Johnson High School,District,"4,761",$3094650.00,$650.00,83.0,84.0,93.27%,97.31%,90.95%
4,Griffin High School,Charter,"1,468",$917500.00,$625.00,83.0,84.0,93.39%,97.14%,90.60%
13,Ford High School,District,"2,739",$1763916.00,$644.00,83.0,84.0,93.87%,96.54%,90.58%
9,Pena High School,Charter,962,$585858.00,$609.00,84.0,84.0,94.59%,95.95%,90.54%


In [149]:
# Sort the data frame by Percent Overall Passing to display the Top 5 Performing Schools
bottom_schools_df = formatted_school_summ_df.sort_values("Percent Overall Passing", ascending=True).head(5)
bottom_schools_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
10,Wright High School,Charter,"1,800",$1049400.00,$583.00,77.0,81.0,66.37%,80.22%,52.99%
2,Shelton High School,Charter,"1,761",$1056600.00,$600.00,77.0,81.0,65.99%,80.74%,53.20%
7,Bailey High School,District,"4,976",$3124928.00,$628.00,77.0,81.0,65.68%,81.32%,53.51%
5,Wilson High School,Charter,"2,283",$1319574.00,$578.00,77.0,81.0,66.75%,80.86%,53.53%
8,Holden High School,Charter,427,$248087.00,$581.00,77.0,81.0,66.06%,81.22%,53.54%


In [158]:
#THIS SECTION WILL GET READING SCORES BY GRADE

#Group student df by both school name and grade and declare it school graded grouped df
school_grade_grouped_df = students_df.groupby(["school_name","grade"])
#use school grouped df to get the mean math score by school per grade
per_mathbygrade = school_grade_grouped_df["math_score"].mean()
#turn the data into a data frame
per_mathbygrade_df =pd.DataFrame(per_mathbygrade)

#Reset the index in a way that the current indexes return as columns (inplace = True)
per_mathbygrade_df.reset_index(inplace = True)

#Rename the columns of the df
per_mathbygrade_df.columns=["School Name", "Grade", "Average Math Score"]

#Need to pivot the data so that the school name becomes the index and grade becomes the column headers (googled)
per_mathbygrade_df = per_mathbygrade_df.pivot(index = "School Name", columns = "Grade",values = "Average Math Score")

#reorder the df so that it looks better
per_mathbygrade_df = per_mathbygrade_df[['9th','10th','11th','12th']]
per_mathbygrade_df

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [159]:
#Repeat for Reading data
per_readbygrade_df = school_grade_grouped_df["reading_score"].mean()
per_readbygrade_df =pd.DataFrame(per_readbygrade_df)

per_readbygrade_df.reset_index(inplace = True)

per_readbygrade_df.columns=["School Name", "Grade", "Average Reading Score"]
per_readbygrade_df = per_readbygrade_df.pivot(index = "School Name", columns = "Grade",values = "Average Reading Score")

per_readbygrade_df = per_readbygrade_df[['9th','10th','11th','12th']]
per_readbygrade_df

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [162]:
#THIS SECTION PUTS INTO CATEGORIES BASED ON SCHOOL SPENDING AND FINDS AVERAGES

#Create bins and categories to classify schools and label
spend_stu_bins = [0,585.9,630.9,645.9,676]
spend_cat =["<$584","$585-629","$630-644","$645-675"]

#use .cut function to add column Spending Ranges to school summary df
school_summary_df["Spending Ranges"] = pd.cut(school_summary_df["Per Student Budget"], bins = spend_stu_bins, labels=spend_cat, include_lowest=True)

#group the data by the Spending Ranges Column
group_avg_spend = school_summary_df.groupby("Spending Ranges")

#Find the average of that grouped data for specified columns
group_avg_spend[['Average Math Score','Average Reading Score','Percent Passing Math','Percent Passing Reading','Percent Overall Passing']].mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
Spending Ranges,,,,,
<$584,78.75,81.75,72.920741,84.639603,62.320523
$585-629,80.25,82.50,79.914839,88.785141,71.964589
$630-644,83.25,84.00,93.800412,96.511302,90.535694
$645-675,79.00,82.00,76.087279,86.180387,66.626727


In [173]:
#THIS SECTION PUTS INTO CATEGORIES BASED ON SCHOOL SIZE AND FINDS AVERAGES

#Create bins and categories to classify schools  by students and label
size_bins = [0,1000,2000,5000]
size_cat =["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

#use .cut function to add column Spending Ranges to school summary df
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], bins = size_bins, labels=size_cat, include_lowest=True)

#group the data by the Spending Ranges Column
group_avg_size = school_summary_df.groupby("School Size")

#Find the average of that grouped data for specified columns
group_avg_size[['Average Math Score','Average Reading Score','Percent Passing Math','Percent Passing Reading','Percent Overall Passing']].mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
School Size,,,,,
Small (<1000),80.5,82.5,80.326073,88.584189,72.039856
Medium (1000-2000),81.0,82.8,82.317324,90.192458,75.270536
Large (2000-5000),80.0,82.5,80.320880,88.769335,72.341377


In [175]:
#THIS SECTION PUTS INTO CATEGORIES BASED ON SCHOOL TYPE AND FINDS AVERAGES

school_summary_df.groupby("School Type").mean()


,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
School Type,,,,,,,,
Charter,1524.250000,9.126881e+05,599.500000,80.375000,82.500000,79.873967,88.624209,71.744987
District,3853.714286,2.478275e+06,643.571429,80.428571,82.714286,82.259154,89.898811,75.029073
